In [0]:
from google.colab import files
import pickle
from IPython.display import Image
uploaded = files.upload()

Saving blosum62.pkl to blosum62.pkl
Saving identity.pkl to identity.pkl
Saving pam250.pkl to pam250.pkl


In [0]:
target = open('pam250.pkl','rb')
substitution_matrix =  pickle.load(target)

In [0]:
target = open('blosum62.pkl','rb')
substitution_matrix =  pickle.load(target)

In [0]:
#Sample Sequences

# seq1 = 'YICSFADCCF' #side column
# seq2 = 'FPCKEECA' #top row
# gap_penalty = 6

seq1= 'ILHFQDLKFBZTY'
seq2= 'YGEKHLVQRERPRCWGSMMDCCAGVM'
gap_penalty = 8


#Global

In [0]:
M = len(seq1)+1 #side column
N = len(seq2)+1 #top row
score_matrix = []
for i in range(M):
    row = []
    score_matrix.append(row)
    for j in range(N):
        row.append(0)

#top row
init_gap_counter = 0
score_matrix[0][0] = 0
for j in range(1,N):
  init_gap_counter -= gap_penalty
  score_matrix[0][j] = init_gap_counter

#Left Column
init_gap_counter = 0
for i in range(1,M):
  init_gap_counter -= gap_penalty
  score_matrix[i][0] = init_gap_counter

def dp_function(row, column):
  E = substitution_matrix[seq1[i-1]][seq2[j-1]]
  diagonal = score_matrix[row-1][column-1] + E
  horizontal = score_matrix[row][column-1] - gap_penalty
  vertical = score_matrix[row-1][column] - gap_penalty
  score_list = [diagonal, vertical, horizontal]

  return max(score_list)
    
for i in range(1,M):
    for j in range(1,N):
        score_matrix[i][j] = dp_function(i,j)

In [0]:
score_matrix

In [0]:
i = M-1
j = N-1
aligned_seq1 = ''
aligned_seq2 = ''

while i != 0 or j != 0:
  if i == 0:
    j = j-1
    aligned_seq1 += '-'
    aligned_seq2 += seq2[j]
    break

  elif j ==0:
    i = i-1
    aligned_seq1 += seq1[i]
    aligned_seq2 += '-'
    break

  cur_score = score_matrix[i][j]
  E = substitution_matrix[seq1[i-1]][seq2[j-1]]
  diag_score = score_matrix[i-1][j-1]
  hori_score = score_matrix[i][j-1]
  vert_score = score_matrix[i-1][j]

  if vert_score - gap_penalty == cur_score:
    aligned_seq1 += seq1[i-1]
    aligned_seq2 += '-'
    i = i-1
    j = j

  elif diag_score + E == cur_score:
    aligned_seq1 += seq1[i-1]
    aligned_seq2 += seq2[j-1]
    i = i-1
    j = j-1

  elif hori_score - gap_penalty == cur_score:
    aligned_seq1 += '-'
    aligned_seq2 += seq2[j-1]
    i = i
    j = j-1

In [0]:
aligned_seq1 = aligned_seq1[::-1] 
aligned_seq2 = aligned_seq2[::-1]

if i > j:
  end_gap_size = i - j 
  pl = ''
  for x in range(end_gap_size):
    pl += '-'
  aligned_seq1 = seq1[0:end_gap_size] + aligned_seq1
  aligned_seq2 = pl + aligned_seq2

elif j > i:
  end_gap_size = j - i
  pl = ''
  for x in range(end_gap_size):
    pl += '-'
  aligned_seq1 = pl + aligned_seq1
  aligned_seq2 = seq2[0:end_gap_size] + aligned_seq2

align_score =  score_matrix[M-1][N-1]      
alignment = (aligned_seq1, aligned_seq2, align_score)

-----KWMQ-CC-Q--R-L---T----
     |     |    |     |    
BQHSZKTFZVLCKRDHRGYDRGTSMAY

Score = -10


#Semiglobal

In [0]:
M = len(seq1)+1 #side column
N = len(seq2)+1 #top row
score_matrix = []
for i in range(M):
    row = []
    score_matrix.append(row)
    for j in range(N):
        row.append(0)


def dp_function(row, column):
  E = substitution_matrix[seq1[i-1]][seq2[j-1]]
  diagonal = score_matrix[row-1][column-1] + E
  horizontal = score_matrix[row][column-1] - gap_penalty
  vertical = score_matrix[row-1][column] - gap_penalty
  score_list = [diagonal, vertical, horizontal]

  return max(score_list)
    
for i in range(1,M):
    for j in range(1,N):
        score_matrix[i][j] = dp_function(i,j)

In [0]:
score_matrix

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, -1, -1, -2, -2, -2, -3, -2, -2, -2, -2, -2, -3, -2, -5, -2, 6],
 [0, 0, 1, 0, 1, 1, -1, -7, -6, 1, 1, 0, 2, -2, -7, -2, -3],
 [0, -2, -2, -1, -3, -2, -3, -2, -8, -8, -2, -2, -4, 0, -8, -10, 2],
 [0, 1, -2, -2, -1, -4, -3, -8, -7, -9, -9, -3, -3, -4, -3, -9, -7],
 [0, -6, -4, -5, -8, -6, -11, -3, -8, -12, -16, -12, -10, -6, -12, -10, -13],
 [0, 0, -4, -3, -2, -5, -5, -12, -7, -5, -9, -14, -8, -9, -11, -9, -13],
 [0, 0, 3, -2, 0, 0, -6, -9, -16, -5, -2, -8, -11, -6, -14, -8, -11],
 [0, -2, 1, 5, -2, -1, -3, -10, -13, -14, -6, -2, -9, -9, -10, -15, -8],
 [0, -4, -7, -1, 0, -7, -6, 4, -3, -12, -15, -10, -8, -11, -13, -15, -15],
 [0, 0, 0, -4, 2, 1, -8, -5, 0, -2, -10, -14, -8, -5, -14, -11, -16],
 [0, 0, 4, 3, -1, 3, 0, -9, -9, 1, 0, -9, -12, -5, -10, -12, -12],
 [0, 0, 3, 6, 6, 1, 2, -4, -13, -7, 4, 1, -6, -10, -10, -7, -14]]

In [0]:
i = M-1
j = N-1
aligned_seq1 = ''
aligned_seq2 = ''

last_row = score_matrix[-1][:]
last_column = []
for x in range(M):
  last_column.append(score_matrix[x][-1])

if max(last_column) == max(last_row):

  column_indices = []
  for z in range(len(last_row)):
    if max(last_column) == last_row[z]:
      column_indices.append(z)
  column_indices.append(N-1)
  j = max(column_indices)
  
  row_indices = []
  for z in range(len(last_column)):
    if  max(last_column) == score_matrix[z][-1]:
      row_indices.append(z)
  i = min(row_indices)

  number_of_gaps = len(seq1)-i

  if number_of_gaps == 0:
    i = i
    j = j
  else:
    aligned_seq1 += seq1[-number_of_gaps:]
    for gap_number in range(number_of_gaps):
      aligned_seq2 += '-'
  align_score = max(last_column)


elif max(last_row) > max(last_column):
  i = M-1
  dupe_indices = []
  for z in range(len(last_row)):
    if last_row[z] == max(last_row):
      dupe_indices.append(z)
  
  j = max(dupe_indices)

  number_of_gaps = len(seq2)-j
  aligned_seq2 += seq2[-number_of_gaps:]
  for gap_number in range(number_of_gaps):
    aligned_seq1 += '-'
  align_score = max(last_row)

    
elif max(last_column) > max(last_row):
  i = last_column.index(max(last_column))
  j = N-1

  number_of_gaps = len(seq1)-i
  aligned_seq1 += seq1[-number_of_gaps:]
  for gap_number in range(number_of_gaps):
    aligned_seq2 += '-'
  align_score = max(last_column)

aligned_seq1 = aligned_seq1[::-1]
aligned_seq2 = aligned_seq2[::-1]

while i != 0 and j != 0:
  cur_score = score_matrix[i][j]
  E = substitution_matrix[seq1[i-1]][seq2[j-1]]

  diag_score = score_matrix[i-1][j-1]
  hori_score = score_matrix[i][j-1]
  vert_score = score_matrix[i-1][j]
  if vert_score - gap_penalty == cur_score:
    aligned_seq1 += seq1[i-1]
    aligned_seq2 += '-'
    i = i-1
    j = j

  elif diag_score + E == cur_score:
    aligned_seq1 += seq1[i-1]
    aligned_seq2 += seq2[j-1]
    i = i-1
    j = j-1

  elif abs(cur_score - hori_score) == gap_penalty:
    aligned_seq1 += '-'
    aligned_seq2 += seq2[j-1]
    i = i
    j = j-1

1


In [0]:
aligned_seq1 = aligned_seq1[::-1] 
aligned_seq2 = aligned_seq2[::-1]

AS1_gap_count = 0
for z in aligned_seq1:
  if z != '-':
    AS1_gap_count += 1

AS2_gap_count = 0
for z in aligned_seq2:
  if z != '-':
    AS2_gap_count += 1

if AS1_gap_count == 0:
  aligned_seq1 = seq1 + aligned_seq1
  gap_string = ''
  for g in range(len(seq1)):
    gap_string += '-'
  aligned_seq2 = gap_string + aligned_seq2
 

elif AS2_gap_count == 0:
  aligned_seq2 = seq2 + aligned_seq2
  gap_string = ''
  for g in range(len(seq2)):
    gap_string += '-'
  aligned_seq1 = gap_string + aligned_seq1

else:
  if i > j:
    end_gap_size = i - j 
    pl = ''
    for x in range(end_gap_size):
      pl += '-'
    aligned_seq1 = seq1[0:end_gap_size] + aligned_seq1
    aligned_seq2 = pl + aligned_seq2

  elif j > i:
    end_gap_size = j - i
    pl = ''
    for x in range(end_gap_size):
      pl += '-'
    aligned_seq1 = pl + aligned_seq1
    aligned_seq2 = seq2[0:end_gap_size] + aligned_seq2

align_score =  align_score       
alignment = (aligned_seq1, aligned_seq2, align_score)

---------------MDLPWDZRFQQZ
               |           
AQHZBGYYBENDHCEM-----------

Score = 6


# Local

In [0]:
M = len(seq1)+1 #side column
N = len(seq2)+1 #top row
score_matrix = []
for i in range(M):
    row = []
    score_matrix.append(row)
    for j in range(N):
        row.append(0)


def dp_function(row, column):
  E = substitution_matrix[seq1[i-1]][seq2[j-1]]
  diagonal = score_matrix[row-1][column-1] + E
  horizontal = score_matrix[row][column-1] - gap_penalty
  vertical = score_matrix[row-1][column] - gap_penalty
  score_list = [diagonal, vertical, horizontal]
  if max(score_list) >= 0:
    return max(score_list)
  elif max(score_list)<0:
    return 0
    
for i in range(1,M):
    for j in range(1,N):
        score_matrix[i][j] = dp_function(i,j)

In [0]:
cur_max = 0
for x in range(1,M):
  if max(score_matrix[x]) > cur_max:
    i = x
    j = score_matrix[x].index(max(score_matrix[x]))
    cur_max = max(score_matrix[x])
align_score = cur_max

max_rows = []
for z in range(len(score_matrix)):
  if align_score in score_matrix[z]:
    max_rows.append(z)

max_row_indices = []
for p in max_rows:
  for t in range(len(score_matrix[p])):
    if score_matrix[p][t] == align_score:
      max_row_indices.append(t)
j = max(max_row_indices)

min_column_indices = []
for d in range(len(score_matrix)):
   if score_matrix[d][j] == align_score:
     min_column_indices.append(d)
i = min(min_column_indices)


aligned_seq1 = ''
aligned_seq2 = ''

while i != 0 or j != 0:
  cur_score = score_matrix[i][j]
  E = substitution_matrix[seq1[i-1]][seq2[j-1]]

  diag_score = score_matrix[i-1][j-1]
  hori_score = score_matrix[i][j-1]
  vert_score = score_matrix[i-1][j]
  if diag_score + E == cur_score:
    aligned_seq1 += seq1[i-1]
    aligned_seq2 += seq2[j-1]
    i = i-1
    j = j-1

  elif vert_score - gap_penalty == cur_score:
    aligned_seq1 += seq1[i-1]
    aligned_seq2 += '-'
    i = i-1
    j = j

  elif abs(cur_score - hori_score) == gap_penalty:
    aligned_seq1 += '-'
    aligned_seq2 += seq2[j-1]
    i = i
    j = j-1

  else:
    break



In [0]:
aligned_seq1 = aligned_seq1[::-1]  
aligned_seq2 = aligned_seq2[::-1]  
align_score =  align_score       
alignment = (aligned_seq1, aligned_seq2, align_score)

FBZTY
     
YGEKH

Score = 10
